In [92]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jun 10 15:23:09 2020

@author: jgonzalezh
"""
import pyodbc
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
import time
from pulp import *
import matplotlib.colors as mcolors
import matplotlib.transforms as mtransforms
import matplotlib.patches as mpatches
import math
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.figure_factory import create_gantt




class DATA:
    def __init__(self,archivo_dmd,archivo_equipos):
        self.Especialidades=archivo_dmd.Especialidad.unique()
        self.Cobertura=archivo_dmd.Tipo.unique() 
        a=len(self.Especialidades)
        b=len(self.Cobertura)
        self.N_Especialidad=a
        self.N_Cobertura=b
        self.Dmd_semanal=np.zeros(shape=(a,b))
        self.Bloquesmax= np.zeros(shape=(a,b))
        for i in range(len(archivo_dmd.index)):
            for e in range(a):
                for c in range(b):
                    if archivo_dmd.iat[i,0]==self.Especialidades[e]:
                        if archivo_dmd.iat[i,1]==self.Cobertura[c]:
                            self.Dmd_semanal[e,c]=archivo_dmd.iat[i,2]
                            self.Bloquesmax[e,c]=archivo_dmd.iat[i,3]
                            
                            
        #Turnos equipos médicos 
        dias=['Lunes','Martes','Miércoles','Jueves','Viernes','Sábado','Domingo'] 
        #Tipo=['Espontaneo', 'Controles']
        horas=['00:00:00','00:30:00','01:00:00','01:30:00','02:00:00','02:30:00','03:00:00','03:30:00',
            '04:00:00','04:30:00','05:00:00','05:30:00','06:00:00','06:30:00','07:00:00','07:30:00',
            '08:00:00','08:30:00','09:00:00','09:30:00','10:00:00','10:30:00','11:00:00','11:30:00',
            '12:00:00','12:30:00','13:00:00','13:30:00','14:00:00','14:30:00','15:00:00','15:30:00',
            '16:00:00','16:30:00','17:00:00','17:30:00','18:00:00','18:30:00','19:00:00','19:30:00',
            '20:00:00','20:30:00','21:00:00','21:30:00','22:00:00','22:30:00','23:00:00','23:30:00']
        FTE=1
        self.turno=archivo_equipos 
        turnos=self.turno
        fill=len(turnos.index)
        
        turnos['Hora Inicio']=turnos['Hora inicio Turno'].dt.hour
        turnos['Minutos Inicio']=turnos['Hora inicio Turno'].dt.minute
        turnos['Hora Fin']=turnos['Hora Fin Turno'].dt.hour
        turnos['Minutos Fin']=turnos['Hora Fin Turno'].dt.minute
        #turnos['Hora Colacion In']=turnos['Hora Inicio Colación'].dt.hour
        #turnos['Minutos colacion In']=turnos['Hora Inicio Colación'].dt.minute
        #turnos['Hora Colacion fin']=turnos['Hora Fin Colación'].dt.hour
        #turnos['Minutos colacion fin']=turnos['Hora Fin Colación'].dt.minute
        self.T=[[[0 for i in range(48)]for j in range(7)]for e in range(self.N_Especialidad)]
        
        for g in range(fill):
            for e in  range(self.N_Especialidad):
                if turnos.iat[g,0]==self.Especialidades[e]:
                    for h in range(7):
                        if turnos.iat[g,2]==dias[h]:
                            for j in range(int(round(2*turnos.iat[g,9]+turnos.iat[g,10]/30))):
                                if j>=int(round(2*turnos.iat[g,7]+turnos.iat[g,8]/30)) and j<=int(round(2*turnos.iat[g,9]+(turnos.iat[g,10]/30))): #revisa si tiene que ponerle -1
                                    self.T[e][h][j]+=FTE

def diccionario_bloques(bloque_ini,bloque_fin):
    diccionario=[]
    h=bloque_ini
    g=bloque_fin
    for k in range(h,g):
        for j in range (k,g):    
            largo=[]
            #for i in [16]:#]range(16,42):
            for u in range(k):
                largo.append(0)
            for i in range(k,g):
                if i<=j:
                    largo.append(1)
                else:
                    largo.append(0)
            for p in range(g,48):
                largo.append(0)
            diccionario.append(largo)
    return diccionario
  

In [94]:
      
DMD=pd.read_excel('C:/Users/jgonzalezh/Documents/Pabellones 2/input/Input_2.xlsx', sheet_name='Horas semanales')
EQ=pd.read_excel('C:/Users/jgonzalezh/Documents/Pabellones 2/input/Horarios médicos_2.xlsx')
#EQ.info()
EQ['Hora inicio Turno']=pd.to_datetime(EQ['Hora inicio Turno'], format='%H:%M:%S')
EQ['Hora Fin Turno']=pd.to_datetime(EQ['Hora Fin Turno'], format='%H:%M:%S')

#sheetname : string, int, mixed list of strings/ints, or None, def
data=DATA(DMD,EQ) 

#data.Dmd_semanal[4,1]
                      
#should make dictionary for variables
E_str=data.Especialidades
C_str=data.Cobertura
E=range(data.N_Especialidad)
C=range(data.N_Cobertura)
P=[0,1,2,3,4,5,6,7,8,9,10]#,11]
P_str=['Pabellón 1','Pabellón 2','Pabellón 3','Pabellón 4','Pabellón 5','Pabellón 6','Pabellón 7','Pabellón 8','UCA 1','UCA 2','UCA 3']
horas=['00:00:00','00:30:00','01:00:00','01:30:00','02:00:00','02:30:00','03:00:00','03:30:00',
    '04:00:00','04:30:00','05:00:00','05:30:00','06:00:00','06:30:00','07:00:00','07:30:00',
    '08:00:00','08:30:00','09:00:00','09:30:00','10:00:00','10:30:00','11:00:00','11:30:00',
    '12:00:00','12:30:00','13:00:00','13:30:00','14:00:00','14:30:00','15:00:00','15:30:00',
    '16:00:00','16:30:00','17:00:00','17:30:00','18:00:00','18:30:00','19:00:00','19:30:00',
    '20:00:00','20:30:00','21:00:00','21:30:00','22:00:00','22:30:00','23:00:00','23:30:00']
dias=[0,1,2,3,4] # se hace el ppl de lunes a viernes
index = np.arange(48) #bloques del día
dias_str=['Lunes','Martes','Miércoles','Jueves','Viernes','Sábado','Domingo'] 

Tipo=[0,1,2,3,4,5,6]
Q_horas=data.Dmd_semanal #ordenadas por especialidad mínimo (vemos a que apuntamos acá)
#Q_portramo=[[14,14,7,21,0,21,7],[15,25,25,10,5,0,5,0],[25,15,25,10,5,0,0,10]]
#Horario de funcionamiento 8:00-21:->16,42
turnos=diccionario_bloques(16,42)
T=range(len(turnos))
#B=[3,4,6]
Cirujanos=data.T
min_sem=data.Bloquesmax
#print(*Cirujanos)


#Definir problema
modelo=LpProblem('Bloques de pabellones', LpMinimize) # Se define el modelo
#Variables
A=LpVariable.dicts('Bloques propuestos', [(e,p,t,d,c)for t in T for d in dias for p in P for e in E for c in C],lowBound=0, upBound=None,cat='Binary')

#Función objetivo        
modelo+=pulp.lpSum([A[(e,p,t,d,c)] for t in T for d in dias for p in P for e in E for c in C]) #minimizar N° Bloques a la semana, Se quiere utilizar al máximo pabellones (maximizar la asignación?)
#Restricciones

# Solo se asigna a una especialidad al mismo tiempo el pabellón
for  p in P:
    for d in dias:
        for h in index:
            modelo+=pulp.lpSum([turnos[t][h]*A[(e,p,t,d,c)] for e in E for t in T for c in C])<=1 
# Se satisface la demanda semanal
for e in E:
    for c in C:
        modelo+=pulp.lpSum([turnos[t][h]*A[(e,p,t,d,c)] for t in T for h in index for d in dias for p in P])>=Q_horas[e,c]*2
# Bloque es continuo abordada por diccionario
# No hay más de tres bloques por día de cada especialidad, ley o VSC
# Acá se puede hacer un vector que contega 1 en la mañana, otro en la tarde del largo de index y se puede castigar en la Fn obj si VSC es en la mañana y si Ley es en la tarde
for d in dias:
    for e in E:
        modelo+=pulp.lpSum([A[(e,p,t,d,c)] for t in T for p in P for c in C])<=3   
#Bloques mínimos semanales, si esq tienen demanda, ROA ley no tiene por ejemplo
for e in E:
    for c in C:
        if Q_horas[e,c]>0:
            modelo+=pulp.lpSum([A[(e,p,t,d,c)] for t in T for d in dias for p in P])>=min_sem[e,c]#B[e] 
# No hay más de tres bloques en simultaneo (abordada por previa, si son numeros distintos se debe hacer)
"""
for d in dias:
    for e in E:
        for h in index:
            modelo+=pulp.lpSum([turnos[t][h]*A[(e,p,t,d)] for t in T for p in P])<=3   
"""
# No hay más bloques que la capacidad de equipos en los bloques ley

for d in dias:
    for e in E:
        for h in index:
            for c in [0]:
                if E_str[e]!='Libre':
                    modelo+=pulp.lpSum([turnos[t][h]*A[(e,p,t,d,c)] for t in T for p in P])<=Cirujanos[e][d][h] #hay que dividr por el promedio de la especialidad o alguna regla   

# No se asignan más pabellones de los disponibles (ya restringidos por N de pabellones).
# A futuro poner largo máximo de un bloque(?) Puede ser en el diccionario de turnos tb para simplificar problema.

for  p in P:
    for d in dias:
        for e in E:
            modelo+=pulp.lpSum([turnos[t][h]*A[(e,p,t,d,c)] for t in T for h in index for c in C])<=18

# Dividiendo el instrumental se puede hacer todas las cirugías a la semana (cajas) --->FUTURA, hay que modelar los datos

modelo.solve(pulp.PULP_CBC_CMD(maxSeconds=25000, msg=0, fracGap=0))#pulp.GLPK()


1

In [95]:
print(LpStatus[modelo.status])
#print(turnos[1][1])

Optimal


In [96]:

Result= [[[[[0 for d in dias] for t in T]for p in P ]for c in C]for e in E]
for e in E:
    for p in P:
        for t in T:
            for d in dias:
                for c in C:
                    Result[e][c][p][t][d]=A[(e,p,t,d,c)].varValue
                    if A[(e,p,t,d,c)].varValue==1:
                        print('E', E_str[e],'C',C_str[c], P_str[p],'T',t,'\nTurno',turnos[t][:],'\nDia',d)



E EESS C Ley Pabellón 1 T 17 
Turno [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
Dia 3
E EESS C VSC Pabellón 2 T 322 
Turno [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0] 
Dia 1
E EESS C Ley Pabellón 3 T 17 
Turno [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
Dia 0
E EESS C Ley Pabellón 3 T 17 
Turno [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
Dia 2
E EESS C Ley Pabellón 4 T 17 
Turno [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
Dia 0
E EESS C Ley Pabellón 4 T 17 
Turno [0, 0, 0, 0, 0, 0, 0, 0

E Libre C VSC Pabellón 6 T 158 
Turno [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0] 
Dia 0
E Libre C Ley Pabellón 6 T 244 
Turno [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0] 
Dia 2
E Libre C VSC Pabellón 7 T 13 
Turno [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
Dia 2
E Libre C Ley Pabellón 7 T 197 
Turno [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0] 
Dia 0
E Libre C Ley UCA 3 T 178 
Turno [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0] 
Dia 0
E Libre C VSC UCA 3 T 197 
Turno [0, 0, 0, 0, 0, 0, 0, 0

In [97]:
import math

#for h in index[1:-1]:
#    modelo
 #!usr/bin/env python
  #coding: utf-8
list=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
def search_ini(list):
    for i in range(48):
        if list[i-1]==0 and list[i]==1:
            z=i
    return z
def search_fin(list):
    for i in range(48):
        if list[i-1]==1 and list[i]==0:
            z=i
    return z
e=search_ini(list)
f=search_fin(list)

def turno(lista):
    #lista=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
    largo=len(lista)
    i=0
    while lista[i]==0:  
        i+=1
    inicio=i
    while lista[i]==1 or lista[i+1]==1:
        fin=i
        i+=1
    if math.modf((inicio)/2)[0] ==0:
        inicio_str=str(int(math.modf((inicio)/2)[1]))+':00:00'
    else:
        inicio_str=str(int(math.modf((inicio)/2)[1]))+':30:00'
    if math.modf((fin)/2)[0] ==0:
        fin_str=str(int(math.modf((fin)/2)[1]))+':30:00'
    else:
        fin_str=str(int(math.modf(((fin)/2)+1)[1]))+':00:00'
    return [inicio_str,fin_str]
                  

In [102]:
def colores(largo):
    colores=['rgb(220, 0, 0)','rgb(220, 55, 0)','rgb(220, 110, 0)','rgb(220, 175, 0)','rgb(220, 220, 0)','rgb(55, 0, 0)',
                'rgb(110, 0, 0)','rgb(220, 0, 0)','rgb(175, 0, 0)','rgb(220, 0, 110)','rgb(220, 55, 110)','rgb(220, 110, 110)',
                'rgb(220, 175, 110)','rgb(220, 220, 110)','rgb(55, 0, 110)','rgb(110, 0, 110)','rgb(220, 0, 110)',
                'rgb(175, 0, 110)','rgb(220, 0, 220)','rgb(220, 55, 220)','rgb(220, 110, 220)',
                'rgb(220, 175, 220)','rgb(220, 220, 220)','rgb(55, 0, 220)','rgb(110, 0, 220)','rgb(220, 0, 220)',
                'rgb(175, 0, 220)']
    return colores[:largo]
df0=[]
df1=[]
df2=[]
df3=[]
df4=[]

resource_0=[]
resource_1=[]
resource_2=[]
resource_3=[]
resource_4=[]
for e in E:
    for p in P:
        for t in T:
            for d in dias:
                for c in C:
                    if Result[e][c][p][t][d]==1:
                        if d==0:

                            resource_0.append(E_str[e]+' '+C_str[c])
                            #df.append(dict(Task=P_str[p], Start=turno(turnos[t])[0],Finish=turno(turnos[t])[1]))
                            df0.append(dict(Task=P_str[p], Start=pd.to_datetime("2020:01:01 "+turno(turnos[t])[0],format='%Y:%m:%d %H:%M:%S'), Finish=pd.to_datetime("2020:01:01 "+turno(turnos[t])[1],format='%Y:%m:%d %H:%M:%S'), Resource=E_str[e]+' '+C_str[c]))       
                        elif d==1:

                            resource_1.append(E_str[e]+' '+C_str[c])
                            #df.append(dict(Task=P_str[p], Start=turno(turnos[t])[0],Finish=turno(turnos[t])[1]))
                            df1.append(dict(Task=P_str[p], Start=pd.to_datetime("2020:01:01 "+turno(turnos[t])[0],format='%Y:%m:%d %H:%M:%S'), Finish=pd.to_datetime("2020:01:01 "+turno(turnos[t])[1],format='%Y:%m:%d %H:%M:%S'), Resource=E_str[e]+' '+C_str[c]))       
                        elif d==2:

                            resource_2.append(E_str[e]+' '+C_str[c])
                            #df.append(dict(Task=P_str[p], Start=turno(turnos[t])[0],Finish=turno(turnos[t])[1]))
                            df2.append(dict(Task=P_str[p], Start=pd.to_datetime("2020:01:01 "+turno(turnos[t])[0],format='%Y:%m:%d %H:%M:%S'), Finish=pd.to_datetime("2020:01:01 "+turno(turnos[t])[1],format='%Y:%m:%d %H:%M:%S'), Resource=E_str[e]+' '+C_str[c]))       
                        elif d==3:

                            resource_3.append(E_str[e]+' '+C_str[c])
                            #df.append(dict(Task=P_str[p], Start=turno(turnos[t])[0],Finish=turno(turnos[t])[1]))
                            df3.append(dict(Task=P_str[p], Start=pd.to_datetime("2020:01:01 "+turno(turnos[t])[0],format='%Y:%m:%d %H:%M:%S'), Finish=pd.to_datetime("2020:01:01 "+turno(turnos[t])[1],format='%Y:%m:%d %H:%M:%S'), Resource=E_str[e]+' '+C_str[c]))       
                        elif d==4:

                            resource_4.append(E_str[e]+' '+C_str[c])
                            #df.append(dict(Task=P_str[p], Start=turno(turnos[t])[0],Finish=turno(turnos[t])[1]))
                            df4.append(dict(Task=P_str[p], Start=pd.to_datetime("2020:01:01 "+turno(turnos[t])[0],format='%Y:%m:%d %H:%M:%S'), Finish=pd.to_datetime("2020:01:01 "+turno(turnos[t])[1],format='%Y:%m:%d %H:%M:%S'), Resource=E_str[e]+' '+C_str[c]))       

#print(len(resource))  
A_0=colores(len(resource_0))
colors_0={}

for i in range(len(resource_0)):
    colors_0[resource_0[i]]=A_0[i]
    
A_1=colores(len(resource_1))
colors_1={}

for i in range(len(resource_1)):
    colors_1[resource_1[i]]=A_1[i]
    
A_2=colores(len(resource_2))
colors_2={}

for i in range(len(resource_2)):
    colors_2[resource_2[i]]=A_2[i]
    
    
A_3=colores(len(resource_3))
colors_3={}

for i in range(len(resource_3)):
    colors_3[resource_3[i]]=A_3[i]
    
    
A_4=colores(len(resource_4))
colors_4={}

for i in range(len(resource_4)):
    colors_4[resource_4[i]]=A_4[i]

In [106]:
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.figure_factory import create_gantt


    
"""colors = {resource[0]: 'rgb(220, 0, 0)',
          resource[1]: (1, 0.9, 0.16),
          resource[2]: 'rgb(0, 255, 100)',
          resource[3]: 'rgb(255, 0, 0)',
          resource[4]: 'rgb(170, 14, 200)',
          resource[5]: (1, 1, 0.2),
          resource[6]: (0, 0.5, 0.7),
          resource[7]: (0, 1, 1),
          resource[8]: (0, 1, 0.5),
          resource[9]: (0.5, 1, 0.7),
          resource[10]: (1, 1, 0.7),
          resource[11]: (0, 0, 0.7),
          resource[12]: (0.1, 0, 0.1),
          resource[13]: (0, 0.3, 0.7),
          resource[14]: (0.5, 0, 0.3),
          resource[15]: (0.1, 0.5, 0.7),
          resource[16]: (0.9, 0.3, 0.7),}
          #resource[17]: (0.9, 1, 0.7)}
"""

fig_0 = ff.create_gantt(df0, colors=colors_0, index_col='Resource', show_colorbar=True, title='Día Lunes',showgrid_x=True, showgrid_y=True,
                      group_tasks=True)
fig_0.show()







In [107]:
fig_1 = ff.create_gantt(df1, colors=colors_1, index_col='Resource', show_colorbar=True, title='Día Martes',showgrid_x=True, showgrid_y=True,
                      group_tasks=True)
fig_1.show()



In [108]:
fig_2 = ff.create_gantt(df2, colors=colors_2, index_col='Resource', show_colorbar=True, title='Día Miércoles',showgrid_x=True, showgrid_y=True,
                      group_tasks=True)
fig_2.show()




In [109]:
fig_3 = ff.create_gantt(df3, colors=colors_3, index_col='Resource', show_colorbar=True, title='Día Jueves',showgrid_x=True, showgrid_y=True,
                      group_tasks=True)
fig_3.show()



In [111]:
fig_4 = ff.create_gantt(df4, colors=colors_4, index_col='Resource', show_colorbar=True, title='Día Viernes',showgrid_x=True, showgrid_y=True,
                      group_tasks=True)
fig_4.show()